In [ ]:
%load_ext autoreload

%autoreload 2

import configparser
import os
import time
import random
import pandas as pd
import requests
import openpyxl
import sqlite3
from sklearn.model_selection import train_test_split

from datetime import datetime
from io import StringIO
from pathlib import Path
from bs4 import BeautifulSoup
from alive_progress import alive_bar
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import OneHotEncoder
from joblib import dump, load

# some stuff I set up in a config file so I don't have to keep updating certain
# variables in every script
config = configparser.ConfigParser()
config.read('../src/config.ini')
output = Path(config['paths']['output'])

In [ ]:
data_df = pd.read_csv(config['data']['model_data'])
upcoming_games = pd.read_csv(r'')

In [8]:
def format_data_for_model(df):
    df.drop(columns=['index','game_date', 'result', 'game_key', 'index.1', 'index.2'], inplace=True)
    df['overall_result'] = df['overall_result'].apply(result_to_num)
    
    df_nums = format_numerical_variables(df)
    df_cats = format_categorical_variables(df)

    formatted_df = pd.concat([df_nums, df_cats],axis=1)
    return formatted_df
    
def format_categorical_variables(df, use_as_encoder=False):
    df_cats = df.select_dtypes(include='O')
    if use_as_encoder:
        encoder = create_encoder(df)
    else:
        encoder = load(output / 'encoder_mvp.joblib')
    df_cats_enc = pd.DataFrame(encoder.transform(df_cats).toarray())
    df_cats_enc.columns = encoder.get_feature_names_out()
    df_cats_enc.index = df_cats.index
    return df_cats_enc
    
def format_numerical_variables(df):
    df_nums = df.select_dtypes(exclude='O')
    return df_nums
    
def create_encoder(df):
    encoder = OneHotEncoder()
    encoder.fit(df)
    return encoder

def result_to_num(result):
    if result == "D":
        return 0
    elif result =="L":
        return -1
    else:
        return 1



In [ ]:
X = data_df.drop(columns=['overall_result', 'home_score', 'home_penalties', 'away_score', 'away_penalties'])
y = data_df[['overall_result', 'home_score', 'home_penalties', 'away_score', 'away_penalties']]
upcoming_games = upcoming_games.drop(columns=['overall_result', 'home_score', 'home_penalties', 'away_score', 'away_penalties'])

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.25, random_state=42)